In [1]:
import ephem
import datetime as DT
import matplotlib.pylab as plt
from numpy.linalg import norm
import matplotlib.animation as animation
from skyfield.positionlib import ICRF
import numpy as  np
import skyfield.sgp4lib as sgp4lib
import skyfield.api as sfapi
from skyfield.api import wgs84, Star
from skyfield.units import Distance
from skyfield.data import hipparcos
import Geoidlib
from scipy.optimize import minimize_scalar
from tangentlib import *

%matplotlib notebook
#%matplotlib qt5
plt.rcParams['figure.figsize'] = [8, 6]
#plt.rcParams['font.size']=26


ModuleNotFoundError: No module named 'tangentlib'

In [ ]:


def loadysb(d):
    ysb=[]
    with open('YBS.edb','r') as fb:
        for line in fb:
            if line[0] !='#' and len(line) >1 : 
                st=ephem.readdb(line)
                st.compute()
                ysb.append(st)
    return ysb

In [2]:
class Mats(object):
    def __init__(self,attime=DT.datetime(2022,1,10,10,0,0)):
        self.date=attime
        self.ts=sfapi.load.timescale()
        self.tle=['1 99988U 22666A   22304.76424065  .00004664  00000-0  44774-3 0 00013',
                  '2 99988 097.6561 307.5659 0012827 298.3476 106.0390 14.93086308000015']
        self.sfmats = sgp4lib.EarthSatellite(self.tle[0],self.tle[1])
        self.period= 2*np.pi/self.sfmats.model.nm
        d=self.date
        t=self.ts.utc(d.year,d.month,d.day,d.hour,d.minute,d.second)
        g=self.sfmats.at(t)
        ECI_pos=g.position.m
        ECI_vel=g.velocity.m_per_s
        self.ECI_pos=ECI_pos
        self.ECI_vel=ECI_vel
        self.t = t
        vunit=np.array(ECI_vel)/norm(ECI_vel)
        mrunit=-np.array(ECI_pos)/norm(ECI_pos)
        yunit=np.cross(mrunit,vunit)
        self.rotmatrix=np.array([vunit,yunit,mrunit]).T 
        self.sublat=g.subpoint().latitude.degrees
        self.sublon=g.subpoint().longitude.degrees
        #print (self.sublat,self.sublon)
        self.ascnod=-np.cross([0,0,1],yunit)
        arg=(np.rad2deg(np.arccos(np.dot(-mrunit,self.ascnod.T))))
        #yaw=-3.3*np.cos(np.deg2rad(arg-np.rad2deg(pitch)-0))
        #yawing not implemented  Needs fraction of orbit
        yaw =0
        yawoffset=0
        pitch=findpitch(92000,t, ECI_pos, np.deg2rad(yaw)+yawoffset, self.rotmatrix)
        #print (np.rad2deg(pitch))
        FOV=rotate(np.array([1,0,0]),np.deg2rad(yaw)+yawoffset,pitch,0,deg=False)
        self.FOV=np.matmul(self.rotmatrix,FOV)
        res = findtangent(t,ECI_pos,self.FOV)
        s=res.x
        #print('s = ', s)
        newp = ECI_pos + (s) * self.FOV
        #print (newp)
        #    pos_s=np.matmul(itrs.rotation_at(t),newp)
        newp=ICRF(Distance(m=newp).au,t=t,center=399)
        self.tplat=(wgs84.subpoint(newp).latitude.degrees)
        self.tplon=(wgs84.subpoint(newp).longitude.degrees)
        newp = ECI_pos + (s-10000) * FOV
        self.tp= (newp)
        #FOV=np.matmul(rotmatrix,FOV)
        v_dash=self.FOV
        [self.FOV_ra,self.FOV_dec]=xyz2radec(v_dash,deg=True)
        r_dash=np.cross(v_dash,yunit)
        self.invrotmatrix=np.linalg.inv(np.array([v_dash,yunit,r_dash]).T) 



        

In [3]:
mats=Mats(DT.datetime(2022,11,10,10,0,0))


NameError: name 'findpitch' is not defined

In [4]:
mats.sublat

NameError: name 'mats' is not defined

In [5]:
st_vec=[]

minmag=2
startdate=DT.datetime(2022,11,30,4,0,0)
d=startdate
t=mats.ts.utc(d.year,d.month,d.day,d.hour,d.minute,d.second)

planets = sfapi.load('de421.bsp')
moon=planets['Moon']
earth=planets['Earth']

#Read in the stars and select only the one we might see and save ECI xyz positions

with sfapi.load.open(hipparcos.URL) as f:
    df = hipparcos.load_dataframe(f)
df=df[df['magnitude']<=2.5]
bright_stars = Star.from_dataframe(df)
nstars=len(bright_stars.ra.hours)
for st in range(nstars): 
    st_vec.append(earth.at(t).observe(bright_stars)[st].position.km)
st_vec.append(earth.at(mats.t).observe(moon).position.km)

ntimes=600    
times=[]
timestamps=[]
st_x=np.zeros((len(st_vec),ntimes))    
st_y=np.zeros((len(st_vec),ntimes)) 
st_ang=np.zeros((len(st_vec),ntimes)) 
for ntime, d in enumerate ([startdate+DT.timedelta(seconds=i*10) for i in range(ntimes)]):
    mats=Mats(d)   
    times.append(d)
    timestamps.append(d.timestamp())
    st_vec.pop(-1) #remove the moon and update
    st_vec.append(earth.at(mats.t).observe(moon).position.km)
    #print('RA = {}, Dec = {}'.format(mats.FOV_ra,mats.FOV_dec))
    for nstar in range(nstars+1): 
        inst_xyz=np.matmul(mats.invrotmatrix,st_vec[nstar])
        [xang,yang]=xyz2radec(inst_xyz,positivera=False)
        st_x[nstar,ntime] = xang
        st_y[nstar,ntime] = yang
        st_ang[nstar,ntime]=np.rad2deg(np.arccos(np.dot(mats.FOV,st_vec[nstar]/norm(st_vec[nstar]))))



    moonpos=earth.at(mats.t).observe(moon).radec()
print (moonpos)
# Treat as extra star  but should realy be in the update part since it moves fast
#could be done there for the correct time but the vector would need to be recalculated
moon_ra = moonpos[0].radians
moon_dec = moonpos[1].radians
print ("Moon RA = {}, Dec= {}".format(moon_ra*180/np.pi,moon_dec*180/np.pi))
#st_vec.append(radec2xyz(moon_ra,moon_dec,deg=False))
#transform to instrument coordinates



NameError: name 'mats' is not defined

In [8]:
possibles=np.array([(istar,itime) for istar in range(nstars+1) for itime in range(ntimes)  
                    if ((st_ang[istar,itime])< 3 and st_y[istar,itime]<0.05 )])

In [9]:
df.loc[df.index[85]]

magnitude              2.480000
ra_degrees           311.551801
dec_degrees           33.969453
parallax_mas          45.260000
ra_mas_per_year      356.160000
dec_mas_per_year     330.280000
ra_hours              20.770120
epoch_year          1991.250000
Name: 102488, dtype: float64

In [10]:
print("Available stars")
print("StarID     Mag       RA         Dec      X-value    Crosstime")
for posstar in np.unique(possibles[:,0]):
    possible=[possible for possible in possibles if possible[0]==posstar ]
    timerange=slice(possible[0][1],possible[-1][1])
    #print (timerange)
    crosstime=DT.datetime.fromtimestamp(np.interp(0,st_y[posstar,timerange][-1::-1],timestamps[timerange][-1::-1]))
    xvalue=np.interp(crosstime.timestamp(),timestamps[timerange],st_x[posstar,timerange])
    if posstar==nstar : 
        mag=-12.6 #full moon 
    else:
        mag=df.loc[df.index[posstar]].magnitude
    print("{:6d} {:7.2f} {:10.3f} {:10.3f}  {:10.5f}  {}".format (posstar, mag,
                                                    Mats(crosstime).FOV_ra,Mats(crosstime).FOV_dec,xvalue, crosstime))

Available stars
StarID     Mag       RA         Dec      X-value    Crosstime
    39    1.67    135.276    -69.259    -0.02010  2022-11-30 04:47:25.920217
    40    2.21    142.661    -59.784     0.03127  2022-11-30 04:44:44.111425
    41    2.47    144.646    -55.649     0.04237  2022-11-30 04:43:35.015878
    44    2.01    158.585     19.417     0.05951  2022-11-30 04:23:06.316992
    45    2.34    167.363     56.165     0.01882  2022-11-30 04:13:07.397376
    46    1.81    169.980     61.290     0.03466  2022-11-30 04:11:42.428446
    86    2.45    -39.083     62.759    -0.01061  2022-11-30 05:33:41.150707
    89    2.07    -15.991    -46.535    -0.04043  2022-11-30 05:04:00.763830
    93  -12.60    -22.033    -14.969    -0.02691  2022-11-30 05:12:29.398240


In [ ]:
nstar

In [ ]:
mats=Mats(crosstime + DT.timedelta(seconds=60))
res=findtangent(mats.t,mats.ECI_pos,st_vec[posstar]/norm(st_vec[posstar]))
funheight(res.x,mats.t,mats.ECI_pos,st_vec[posstar]/norm(st_vec[posstar]))

In [ ]:
plt.figure()
plt.plot(times[timerange],st_y[posstar,timerange],times[timerange],st_x[posstar,timerange],times[timerange],st_ang[posstar,timerange]/100)

In [ ]:
(DT.datetime.fromtimestamp(np.interp(0,st_y[posstar,timerange][-1::-1],timestamps[timerange][-1::-1])))

In [ ]:
possibles

In [ ]:
np.array(times).timestamp()

In [11]:
from scipy.spatial.transform import Rotation as R

In [24]:
newrot=R.align_vectors(np.array([1,1,1],ndmin=2),np.array([0,1,1],ndmin=2))

/var/folders/z9/z5mq6p390ls748hn4fyfy9pw0000gn/T/ipykernel_36217/246850017.py:1: UserWarning: Optimal rotation is not uniquely or poorly defined for the given sets of vectors.
  newrot=R.align_vectors(np.array([1,1,1],ndmin=2),np.array([0,1,1],ndmin=2))


In [54]:
newrot[0].inv().apply([1,1,1])

array([-5.55111512e-17,  1.22474487e+00,  1.22474487e+00])

In [40]:
np.array([1,1,1],ndmin=2).shape

(1, 3)

In [55]:
newrot[0].as_quat

array([-3.27642678e-17,  2.14186495e-01, -2.14186495e-01,  9.53020614e-01])

In [41]:
def rotation_matrix_from_vectors(vec1, vec2):
    """ Find the rotation matrix that aligns vec1 to vec2
    :param vec1: A 3d "source" vector
    :param vec2: A 3d "destination" vector
    :return mat: A transform matrix (3x3) which when applied to vec1, aligns it with vec2.
    """
    a, b = (vec1 / np.linalg.norm(vec1)).reshape(3), (vec2 / np.linalg.norm(vec2)).reshape(3)
    v = np.cross(a, b)
    c = np.dot(a, b)
    s = np.linalg.norm(v)
    kmat = np.array([[0, -v[2], v[1]], [v[2], 0, -v[0]], [-v[1], v[0], 0]])
    rotation_matrix = np.eye(3) + kmat + kmat.dot(kmat) * ((1 - c) / (s ** 2))
    return rotation_matrix

In [44]:
rott=rotation_matrix_from_vectors(np.array([1,1,1]),np.array([0,1,1]))

In [50]:
np.matmul(rott,[1,1,1])

array([0.        , 1.22474487, 1.22474487])